In [1]:
import numpy as np
import pandas as pd

Edit-distance operations are defined as 
* (a,a) denotes a match of symbols at the given position
* (a,-) denotes deletion of symbol 'a' at some position
* (-,b) denotes insertion of symbol 'b' at some position
* (a,b) denotes replacement of 'a' with 'b' at some position, and a != b
We assign a separate cost for each of these operations according to our needs

In [2]:
#for simplicity and completeness sake, we define a simple levenshtein distance function first

def basic_distance (trace1, trace2):

    M = len(trace1)
    N = len(trace2)
    edit_table = np.zeros((M,N)) #establish table

    #fill table
    for i in range(M):
        for j in range(N):

            if i == 0:
                edit_table[i][j] = j

            elif j ==0:
                edit_table[i][j] = i

            elif trace1[i-1] == trace2[j-1]:
                edit_table[i][j] = edit_table[i-1][j-1]

            else:
                edit_table[i][j] = 1 + min(edit_table[i-1][j], edit_table[i][j-1], edit_table[i-1][j-1]) #scoring done here

    return edit_table[i][j]

In [3]:
#sample test
dat1 = pd.read_csv('./data/graham.norton.s22.e08_data.csv')
dat2 = pd.read_csv('./data/graham.norton.s22.e12_data.csv')
dat3 = pd.read_csv('./data/blackpink_data.csv')
test1 = list(dat1.L)
test2 = list(dat2.L)
test3 = list(dat3.L)

In [4]:
basic_distance(test1,test2)

670.0

In [5]:
a = ["a","a","a","a","a"]
c = ["a","a","a","a","a"]
b = ["b","b","b","b","b"]
d = ["a","a","a","a","a","a","a","a"]
basic_distance(a,d)

3.0

In [6]:
a = ["a","a","a","a","b","a","c","c"]
b = ["b","b","b","b","a","b","c","c","c","c"]
basic_distance(a,b)

6.0

In [7]:
basic_distance(test1, test3)

991.0

In [8]:
#baseline
import nltk
nltk.edit_distance(test1,test2)

670

In [9]:
t1 = ["a","b"]
t2 = ["a","b","c","d"]
nltk.edit_distance(t1,t2)

2

# Now we have to alter the function to accomodate for different scoring metrics according to the paper
* substitution of uncorrelated activities should be discouraged
* substitution of contrasting activities should be penalized
* insertion of activities out of context should be discouraged
* substitution of correlated activities should be encouraged in proportion to the degree of similarity

In [10]:
#STEP 1: Define the symbols in the list of traces
def define_symbols (traces):
    assert type(traces) == list
    symbols = []
    for item in traces:
        symbols.append(set(item))
    x = symbols[0]
    for i in range(len(symbols)):
        x = x.union(symbols[i])
    
    return list(x)

In [11]:
# A != B
a = ["a","b","c","d","e"]
b = ["a","g","q","q","e","f"]
c = [a, b]
print(define_symbols(c))

# A == B
a = ["a", "b", "c", "d"]
b = ["a", "a", "c", "d", "b", "b"]
c = [a,b]
print(define_symbols(c))

# on the dataset
c = [test1,test2]
print(define_symbols(c))

['e', 'b', 'd', 'g', 'q', 'f', 'c', 'a']
['b', 'd', 'c', 'a']
['give.opinion', 'give.statement', 'relax.atmosphere', 'closed.question', 'recall', 'respond.deny', 'exclamation', 'respond.agree', 'open.question', 'use.social.convention', 'misc', 'quoted.speech', 'deflection']


In [12]:
#STEP 2: Define the set of all 3-grams in the logs and their frequencies
def three_grams (traces):
    assert type(traces) == list
    g3 = []
    g3_freq = {}
    for trace in traces:
        for i in range(len(trace)-2):
            g3.append(", ".join(list(trace[i:i+3])))
            try:
                g3_freq[", ".join(list(trace[i:i+3]))] += 1
            except:
                g3_freq[", ".join(list(trace[i:i+3]))] = 1
    return list(set(g3)), g3_freq

In [13]:
#A = abcdefghijk, B = kjihgfedcba
a = ["a","b","c","d","e","f","g","h","i","j","k"]
b = ["k","j","i","h","g","f","e","d","c","b","a"]
c = [a,b]
print(three_grams(c))

#A = abcdefghijk, B = abcdefghijk
a = ["a","b","c","d","e","f","g","h","i","j","k"]
b = ["a","b","c","d","e","f","g","h","i","j","k"]
c = [a,b]
print(three_grams(c))

(['g, f, e', 'j, i, h', 'b, c, d', 'e, f, g', 'k, j, i', 'h, g, f', 'f, e, d', 'a, b, c', 'i, h, g', 'd, c, b', 'f, g, h', 'c, b, a', 'e, d, c', 'c, d, e', 'g, h, i', 'h, i, j', 'i, j, k', 'd, e, f'], {'a, b, c': 1, 'b, c, d': 1, 'c, d, e': 1, 'd, e, f': 1, 'e, f, g': 1, 'f, g, h': 1, 'g, h, i': 1, 'h, i, j': 1, 'i, j, k': 1, 'k, j, i': 1, 'j, i, h': 1, 'i, h, g': 1, 'h, g, f': 1, 'g, f, e': 1, 'f, e, d': 1, 'e, d, c': 1, 'd, c, b': 1, 'c, b, a': 1})
(['b, c, d', 'e, f, g', 'a, b, c', 'f, g, h', 'c, d, e', 'g, h, i', 'h, i, j', 'i, j, k', 'd, e, f'], {'a, b, c': 2, 'b, c, d': 2, 'c, d, e': 2, 'd, e, f': 2, 'e, f, g': 2, 'f, g, h': 2, 'g, h, i': 2, 'h, i, j': 2, 'i, j, k': 2})


In [14]:
#STEP 3: Define the context for symbol a
def define_context(grams):
    
    assert type(grams) == list
    
    context = {}
    for gram in grams:
        x,a,y = gram.split(", ")
        try:
            context[a].append("{0}, {1}".format(x,y))
        except:
            context[a] = []
            context[a].append("{0}, {1}".format(x,y))
            
    #clear dups
    for k in list(context.keys()):
        context[k] = list(set(context[k]))
    
    return context

In [15]:
#test context, should have keys a and b, where each has aa and bb respectively
a = ["a","a","a","a","a","a","a"]
b = ["b","b","b","b","b"]
c = [a,b]
grams, g3_freq = three_grams(c)
print(define_context(grams))

#test context
a = ["a","a","a","a","a","a","a","c","e","f","a","a","b","c"]
b = ["b","b","b","b","b","c","c","a"]
c = [a,b]
grams, g3_freq = three_grams(c)
print(define_context(grams))

{'a': ['a, a'], 'b': ['b, b']}
{'c': ['c, a', 'a, e', 'b, c'], 'e': ['c, f'], 'b': ['a, c', 'b, c', 'b, b'], 'a': ['a, b', 'a, c', 'a, a', 'f, a'], 'f': ['e, a']}


In [16]:
#STEP 4: define pairs of context
def context_pairs (context):
    
    assert type(context) == dict
    
    context_pairs = {}
    for a in list(context.keys()):
        for b in list(context.keys()):
            if a != b:
                context_pairs["{0}, {1}".format(a, b)] = list(set(context[a]).intersection(set(context[b])))
    
    return context_pairs

In [17]:
#test context pairs, should be empty
a = ["a","a","a","a","a","a","a"]
b = ["b","b","b","b","b"]
c = [a,b]
grams, g3_freq = three_grams(c)
context = define_context(grams)
print(context_pairs(context))

#test context pairs, should be aa and bb
a = ["a","a","a","a","b","a"]
b = ["b","b","b","b","a","b"]
c = [a,b]
grams, g3_freq = three_grams(c)
context = define_context(grams)
print(context_pairs(context))

{'a, b': [], 'b, a': []}
{'a, b': ['a, a', 'b, b'], 'b, a': ['a, a', 'b, b']}


In [18]:
#STEP 5: define co-occurrence combinations
def define_cooccurrence(symbols, context_pairs, gram_freq):
    
    assert type(context_pairs) == dict
    assert type(gram_freq) == dict
    assert type(symbols) == list
    
    co_occur = {}
    for k in list(context_pairs.keys()):
        for item in context_pairs[k]:
            for a in symbols:
                for b in symbols:
                    x,y = item.split(", ")[0], item.split(", ")[1]
                    if a == b:
                        try:
                            n = gram_freq["{0}, {1}, {2}".format(x,a,y)]
                            co_occur["{0}, {1}({2}, {3})".format(x,y,a,b)] = (n*(n-1))/2
                        except:
                            co_occur["{0}, {1}({2}, {3})".format(x,y,a,b)] = 0.0
                        
                    elif a != b:
                        try:
                            n_i = gram_freq["{0}, {1}, {2}".format(x,a,y)]
                            n_j = gram_freq["{0}, {1}, {2}".format(x,b,y)]
                            co_occur["{0}, {1}({2}, {3})".format(x,y,a,b)] = n_i*n_j
                        except:
                            co_occur["{0}, {1}({2}, {3})".format(x,y,a,b)] = 0.0
    
    return co_occur
            

In [19]:
#test
a = ["a","a","a","a","b","a","c"]
b = ["b","b","b","b","a","b","c"]
c = [a,b]
grams, g3_freq = three_grams(c)
print(grams)
print(g3_freq)
context = define_context(grams)
print(context)
con_pairs = context_pairs(context)
print(con_pairs)
print(define_cooccurrence(define_symbols(c),con_pairs,g3_freq))

['a, a, b', 'b, b, a', 'a, b, c', 'b, a, c', 'a, b, a', 'a, a, a', 'b, a, b', 'b, b, b']
{'a, a, a': 2, 'a, a, b': 1, 'a, b, a': 1, 'b, a, c': 1, 'b, b, b': 2, 'b, b, a': 1, 'b, a, b': 1, 'a, b, c': 1}
{'a': ['a, b', 'a, a', 'b, c', 'b, b'], 'b': ['b, a', 'a, c', 'a, a', 'b, b']}
{'a, b': ['a, a', 'b, b'], 'b, a': ['a, a', 'b, b']}
{'a, a(b, b)': 0.0, 'a, a(b, c)': 0.0, 'a, a(b, a)': 2, 'a, a(c, b)': 0.0, 'a, a(c, c)': 0.0, 'a, a(c, a)': 0.0, 'a, a(a, b)': 2, 'a, a(a, c)': 0.0, 'a, a(a, a)': 1.0, 'b, b(b, b)': 1.0, 'b, b(b, c)': 0.0, 'b, b(b, a)': 2, 'b, b(c, b)': 0.0, 'b, b(c, c)': 0.0, 'b, b(c, a)': 0.0, 'b, b(a, b)': 2, 'b, b(a, c)': 0.0, 'b, b(a, a)': 0.0}


In [20]:
#STEP 6: Define the count of co-occurrences for symbols a,b for all contexts
def co_occur_combos(symbols, con_pairs, co_occurs):
    assert type(symbols) == list
    assert type(con_pairs) == dict
    assert type(co_occurs) == dict
    
    co_occur_combos = {}
    for a in symbols:
        for b in symbols:
            total = 0.0
            for k in list(con_pairs.keys()):
                for item in con_pairs[k]:
                    total += co_occurs["{0}({1}, {2})".format(item,a,b)]
            co_occur_combos["{0}, {1}".format(a,b)] = total
    
    return co_occur_combos


In [21]:
#test
a = ["a","a","a","a","b","a","a","b","b"]
b = ["a","a","a","a","b","a","a","b","a","a","b","a","a","b","a","a","b","a"]
c = [a,b]
grams, g3_freq = three_grams(c)
print(g3_freq)
context = define_context(grams)
print(context)
con_pairs = context_pairs(context)
print(con_pairs)
co_occurs = define_cooccurrence(define_symbols(c),con_pairs,g3_freq)
print(co_occurs)
print(co_occur_combos(define_symbols(c),con_pairs,co_occurs))

{'a, a, a': 4, 'a, a, b': 7, 'a, b, a': 6, 'b, a, a': 5, 'a, b, b': 1}
{'a': ['a, b', 'b, a', 'a, a'], 'b': ['a, b', 'a, a']}
{'a, b': ['a, b', 'a, a'], 'b, a': ['a, b', 'a, a']}
{'a, b(b, b)': 0.0, 'a, b(b, a)': 7, 'a, b(a, b)': 7, 'a, b(a, a)': 21.0, 'a, a(b, b)': 15.0, 'a, a(b, a)': 24, 'a, a(a, b)': 24, 'a, a(a, a)': 6.0}
{'b, b': 30.0, 'b, a': 62.0, 'a, b': 62.0, 'a, a': 54.0}


In [22]:
#STEP 7: Define norm on the count of co-occur combos
def define_norm (co_combos):
    assert type(co_combos) == dict
    norm = 0.0
    for k in list(co_combos.keys()):
        norm += co_combos[k]
    
    return norm

In [23]:
#test
a = ["a","a","a","a","b","a","a","b","b"]
b = ["a","a","a","a","b","a","a","b","a","a","b","a","a","b","a","a","b","a"]
c = [a,b]
grams, g3_freq = three_grams(c)
context = define_context(grams)
con_pairs = context_pairs(context)
co_occurs = define_cooccurrence(define_symbols(c),con_pairs,g3_freq)
print(define_norm(co_occur_combos(define_symbols(c),con_pairs,co_occurs)))

208.0


In [24]:
#STEP 8: Define matrix M over A x A
def define_matrix (symbols, co_combos, norm):
    assert type(symbols) == list
    assert type(co_combos) == dict
    assert type(norm) == float
    
    mat_M = {}
    for a in symbols:
        for b in symbols:
            mat_M["{0}, {1}".format(a,b)] = co_combos["{0}, {1}".format(a,b)]/norm
    
    return mat_M

In [25]:
a = ["a","a","a","a","b","a","a","b","b"]
b = ["a","a","a","a","b","a","a","b","a","a","b","a","a","b","a","a","b","a"]
c = [a,b]
grams, g3_freq = three_grams(c)
context = define_context(grams)
con_pairs = context_pairs(context)
co_occurs = define_cooccurrence(define_symbols(c),con_pairs,g3_freq)
norm = define_norm(co_occur_combos(define_symbols(c),con_pairs,co_occurs))
co_combos = co_occur_combos(define_symbols(c), con_pairs, co_occurs)
print(define_matrix(define_symbols(c), co_combos, norm))

{'b, b': 0.14423076923076922, 'b, a': 0.2980769230769231, 'a, b': 0.2980769230769231, 'a, a': 0.25961538461538464}


In [26]:
def prob_occur (symbols, mat_M):
    assert type(symbols) == list
    assert type(mat_M) == dict
    
    p = {}
    for a in symbols:
        total = 0
        for b in symbols:
            if a != b:
                total += mat_M["{0}, {1}".format(a,b)]
        total += mat_M["{0}, {1}".format(a,a)]
        p["{0}".format(a)] = total
    
    return p
                

In [27]:
a = ["a","a","a","a","b","a","a","b","b"]
b = ["a","a","a","a","b","a","a","b","a","a","b","a","a","b","a","a","b","a"]
c = [a,b]
grams, g3_freq = three_grams(c)
context = define_context(grams)
con_pairs = context_pairs(context)
co_occurs = define_cooccurrence(define_symbols(c),con_pairs,g3_freq)
norm = define_norm(co_occur_combos(define_symbols(c),con_pairs,co_occurs))
co_combos = co_occur_combos(define_symbols(c), con_pairs, co_occurs)
matM = define_matrix(define_symbols(c), co_combos, norm)
print(prob_occur(define_symbols(c), matM))

{'b': 0.4423076923076923, 'a': 0.5576923076923077}


In [28]:
def exp_val (symbols, prob):
    assert type(symbols) == list
    assert type(prob) == dict
    
    e_val = {}
    for a in symbols:
        for b in symbols:
            if a == b:
                e_val["{0}, {1}".format(a,b)] = prob["{0}".format(a)]**2
            else:
                e_val["{0}, {1}".format(a,b)] = 2*prob["{0}".format(a)]*prob["{0}".format(b)]
    
    return e_val

In [29]:
a = ["a","a","a","a","b","a","a","b","b"]
b = ["a","a","a","a","b","a","a","b","a","a","b","a","a","b","a","a","b","a"]
c = [a,b]
grams, g3_freq = three_grams(c)
context = define_context(grams)
con_pairs = context_pairs(context)
co_occurs = define_cooccurrence(define_symbols(c),con_pairs,g3_freq)
norm = define_norm(co_occur_combos(define_symbols(c),con_pairs,co_occurs))
co_combos = co_occur_combos(define_symbols(c), con_pairs, co_occurs)
matM = define_matrix(define_symbols(c), co_combos, norm)
probs = prob_occur(define_symbols(c), matM)
print(exp_val(define_symbols(c), probs))

{'b, b': 0.1956360946745562, 'b, a': 0.4933431952662722, 'a, b': 0.4933431952662722, 'a, a': 0.3110207100591716}


In [30]:
def sub_scores (traces):
    assert type(traces) == list
    
    symbols = define_symbols(traces)
    three_gs, three_gs_freq = three_grams(traces)
    cons = define_context(three_gs)
    con_pairs = context_pairs(cons)
    co_occurs = define_cooccurrence(symbols, con_pairs, three_gs_freq)
    co_combos = co_occur_combos(symbols, con_pairs, co_occurs)
    norm = define_norm(co_combos)
    matM = define_matrix(symbols, co_combos, norm)
    probs = prob_occur(symbols, matM)
    e_val = exp_val(symbols, probs)
    
    sub_costs = {}
    for a in symbols:
        for b in symbols:
            if a!=b:
                try:
                    sub_costs["{0}, {1}".format(a,b)] = np.log2(matM["{0}, {1}".format(a,b)]/e_val["{0}, {1}".format(a,b)])
                except:
                    sub_costs["{0}, {1}".format(a,b)] = -1000
    
    return sub_costs
    

In [31]:
s_score = sub_scores([test1, test2,test3])
s_score

{'give.opinion, give.statement': -1.551085076399558,
 'give.opinion, relax.atmosphere': -1.3650380400618178,
 'give.opinion, closed.question': -1.139522845910327,
 'give.opinion, x': -0.6679891654646126,
 'give.opinion, recall': -1.1929744100425526,
 'give.opinion, respond.deny': -1.0729193484268291,
 'give.opinion, exclamation': -0.7793724385005201,
 'give.opinion, respond.agree': -1.0515448148414683,
 'give.opinion, open.question': -1.5235479035656052,
 'give.opinion, use.social.convention': -1.587120755005616,
 'give.opinion, misc': -0.8011931628395911,
 'give.opinion, quoted.speech': -0.548279866499742,
 'give.opinion, deflection': -0.7705444199074922,
 'give.statement, give.opinion': -1.551085076399558,
 'give.statement, relax.atmosphere': 0.161794917912701,
 'give.statement, closed.question': -0.49663267156671204,
 'give.statement, x': -1.4430844811671424,
 'give.statement, recall': -2.285814647337215,
 'give.statement, respond.deny': -0.774043474918979,
 'give.statement, exclama

In [32]:
#gut check
print(sub_scores([test1, test2]) == sub_scores([test2, test1]))

/home/kense/.local/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in log2


False


# Note that with very broad categories of labels, there is going to be little defined similarity between each label, and switching the labels will always be bad (thus negative score for all substitutions), however this is expected, and we can see something like, what are the least harmful switches, to show similarity. Notice closed.question and open.question have higher similarity, which is something expected

In [33]:
#Insertion STEP 4: define Cxy(a) as the count of occurrences of 3-gram xay
def occ_count (symbols, cons, grams, gfreq):
    assert type(symbols) == list
    assert type(grams) == list
    assert type(cons) == dict
    
    o_counts = {}
    for a in list(cons.keys()):
        for pair in cons[a]:
            x = pair.split(", ")[0]
            y = pair.split(", ")[1]
            o_counts["{0}, {1}({2})".format(x,y,a)] = gfreq["{0}, {1}, {2}".format(x,a,y)]
    
    return o_counts

In [34]:
#test occ_count with values easily verifiable
t1 = ["a","a","a","a","a"]
t2 = ["a","a","a","a","b"]
tlog = [t1, t2]
symbols = define_symbols(tlog)
grams, gfreq = three_grams(tlog)
cons = define_context(grams)
oc = occ_count(symbols, cons, grams, gfreq)
print(oc)

{'a, b(a)': 1, 'a, a(a)': 5}


In [35]:
#Insertion STEP 5: define countRgivenL
def countRgL (symbols, ocounts):
    assert type(symbols) == list
    assert type(ocounts) == dict
    
    rgl_counts = {}
    
    for a in symbols:
        for x in symbols:
            #if a !=x:
            total = 0
            for k in list(ocounts.keys()):
                if k.split("(")[0].split(", ")[0] == x and k.split("(")[1] == "{0})".format(a):
                    total += ocounts[k]
            rgl_counts["{0}/{1}".format(a,x)] = total
    
    return rgl_counts

In [36]:
#test with values easily verifiable
t1 = ["a","a","a","a","a","a"]
t2 = ["a","a","a","a","b","a"]
tlog = [t1, t2]
symbols = define_symbols(tlog)
grams, gfreq = three_grams(tlog)
cons = define_context(grams)
oc = occ_count(symbols, cons, grams, gfreq)
rgl = countRgL(symbols, oc)
print(rgl)

#should add c to the results table, but no occurrences since it's at the end, opens 1 more for a/b
t1 = ["a","a","a","a","a","a","c"]
t2 = ["a","a","a","a","b","a","c"]
tlog = [t1, t2]
symbols = define_symbols(tlog)
grams, gfreq = three_grams(tlog)
cons = define_context(grams)
oc = occ_count(symbols, cons, grams, gfreq)
rgl = countRgL(symbols, oc)
print(rgl)

{'b/b': 0, 'b/a': 1, 'a/b': 0, 'a/a': 7}
{'b/b': 0, 'b/c': 0, 'b/a': 1, 'c/b': 0, 'c/c': 0, 'c/a': 0, 'a/b': 1, 'a/c': 0, 'a/a': 8}


In [37]:
#Insertion STEP 6: define norm(a)
def rgl_norm (symbols, rgl_counts):
    assert type(symbols) == list
    assert type(rgl_counts) == dict
    
    rgl_norms = {}
    
    for a in symbols:
        total = 0
        for x in symbols:
            #if a !=x:
            total += rgl_counts["{0}/{1}".format(a,x)]
        rgl_norms["{0}".format(a)] = total
    
    return rgl_norms

In [38]:
#test with values easily verifiable
t1 = ["a","a","a","a","a","a"]
t2 = ["a","a","a","a","b","a"]
tlog = [t1, t2]
symbols = define_symbols(tlog)
grams, gfreq = three_grams(tlog)
cons = define_context(grams)
oc = occ_count(symbols, cons, grams, gfreq)
rgl = countRgL(symbols, oc)
norms = rgl_norm(symbols, rgl)
print(norms)

{'b': 1, 'a': 7}


In [39]:
#Insertion STEP 7: define the probability of all symbols
def rgl_prob (trace):
    assert type(trace) == list
    
    p = {}
    for item in trace:
        for a in item:
            try:
                p["{0}".format(a)] += 1
            except:
                p["{0}".format(a)] = 1
    
    tot_len = 0
    for item in trace:
        tot_len += len(item)
    
    for k in list(p.keys()):
        p[k] = p[k]/tot_len
    
    return p

In [40]:
#test with values easily verifiable
t1 = ["a","a","a","a","a"]
t2 = ["a","a","a","a","b"]
tlog = [t1, t2]
symbols = define_symbols(tlog)
grams, gfreq = three_grams(tlog)
cons = define_context(grams)
oc = occ_count(symbols, cons, grams, gfreq)
rgl = countRgL(symbols, oc)
norms = rgl_norm(symbols, rgl)
probs = rgl_prob(tlog)
print(probs)

{'a': 0.9, 'b': 0.1}


In [41]:
#Insertion STEP 8: define rglNorm
def normed_counts (symbols, rgl, norms):
    assert type(symbols) == list
    assert type(rgl) == dict
    assert type(norms) == dict
    
    normed_rgls = {}
    
    for a in symbols:
        for b in symbols:
            normed_rgls["{0}/{1}".format(a,b)] = rgl["{0}/{1}".format(a,b)]/norms["{0}".format(a)]
    
    return normed_rgls

In [42]:
#test with values easily verifiable
t1 = ["a","a","a","a","a","a"]
t2 = ["a","a","a","a","b","a"]
tlog = [t1, t2]
symbols = define_symbols(tlog)
grams, gfreq = three_grams(tlog)
cons = define_context(grams)
oc = occ_count(symbols, cons, grams, gfreq)
rgl = countRgL(symbols, oc)
norms = rgl_norm(symbols, rgl)
probs = rgl_prob(tlog)
norm_rgls = normed_counts(symbols, rgl, norms)
print(norm_rgls)

{'b/b': 0.0, 'b/a': 1.0, 'a/b': 0.0, 'a/a': 1.0}


In [43]:
def insert_scores (traces):
    assert type(traces) == list
    
    symbols = define_symbols(traces)
    grams, freq = three_grams(traces)
    cons = define_context(grams)
    oc = occ_count(symbols, cons, grams, freq)
    rgl = countRgL(symbols, oc)
    norms = rgl_norm(symbols, rgl)
    probs = rgl_prob(traces)
    norm_rgls = normed_counts(symbols ,rgl, norms)
    
    scores = {}
    for a in symbols:
        for b in symbols:
            scores["{0}/{1}".format(a,b)] = np.log2(norm_rgls["{0}/{1}".format(a,b)]/probs["{0}".format(a)]*probs["{0}".format(b)])
    
    #replace -inf
    for k in list(scores.keys()):
        if scores[k] == -np.inf:
            scores[k] = -1000
    
    return scores

In [44]:
t1 = ["a","a","a","a","a","a"]
t2 = ["a","a","a","a","a","a"]
tlog = [t1, t2]
print(insert_scores(tlog))

t1 = ["a","a","a","a","a","a"]
t2 = ["a","a","a","a","b","a"]
tlog = [t1, t2]
print(insert_scores(tlog))

t1 = ["a","a","b","a","a","b","c","c"]
t2 = ["a","a","b","a","a","b","a","a"]
tlog = [t1, t2]
print(insert_scores(tlog))

{'a/a': 0.0}
{'b/b': -1000, 'b/a': 3.4594316186372973, 'a/b': -1000, 'a/a': 0.0}
{'b/b': -1000, 'b/c': -1000, 'b/a': 1.3219280948873624, 'c/b': 1.0, 'c/c': -1000, 'c/a': -1000, 'a/b': -2.5443205162238103, 'a/c': -1000, 'a/a': -0.8073549220576042}


/home/kense/.local/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log2
  app.launch_new_instance()


# Here this seems correct since adding b given a will make t1 more similar to t2, where the only difference is that t2 has an extra b!

In [45]:
#test on our logs, specifically looking for respond.agree, open.question or respond.agree, closed.question to have high scores, since they usually
#appear next to each other in that order
ins_score = insert_scores([test1,test2,test3])
ins_score

/home/kense/.local/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log2
  app.launch_new_instance()


{'give.opinion/give.opinion': -0.7252058807037325,
 'give.opinion/give.statement': -4.9083757174350895,
 'give.opinion/relax.atmosphere': -6.088210604978642,
 'give.opinion/closed.question': -8.540568381362702,
 'give.opinion/x': -11.949373926930031,
 'give.opinion/recall': -4.381951667931271,
 'give.opinion/respond.deny': -11.31550182572793,
 'give.opinion/exclamation': -11.444784842672895,
 'give.opinion/respond.agree': -6.037753179409116,
 'give.opinion/open.question': -9.222392421336448,
 'give.opinion/use.social.convention': -5.80453990645055,
 'give.opinion/misc': -7.467504919172979,
 'give.opinion/quoted.speech': -15.029747343394051,
 'give.opinion/deflection': -12.942284502143712,
 'give.statement/give.opinion': -1.7300799373441995,
 'give.statement/give.statement': -1.2152326297635818,
 'give.statement/relax.atmosphere': -4.294389567337964,
 'give.statement/closed.question': -6.672746762278248,
 'give.statement/x': -11.88890722990318,
 'give.statement/recall': -5.2230812668434

# scores generally make sense

# we can define similarity function

In [46]:
#gutcheck
insert_scores([test1,test2]) == insert_scores([test2, test1])

/home/kense/.local/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log2
  app.launch_new_instance()


True

In [47]:
def convert_cost(cost):
    #convert cost to a distance penalty
    pass

In [48]:
def calc_similarity(trace1, trace2, sub_cost, ins_cost, probs):
    
    assert type(trace1) == type(trace2) == list
    
    #pad traces
    trace1 = ["_"] + trace1
    trace2 = ["_"] + trace2
    
    #set shorter one as tr1
    if len(trace1) > len(trace2):
        copy = trace1
        trace1 = trace2
        trace2 = copy

    M = len(trace1)
    N = len(trace2)
    sim_table = np.zeros((M,N)) #establish table
    s_score = sub_cost #get substitution score
    ins_score = ins_cost #get insertion score
    p = probs #get probabilities
    
    #fill table, horizontal -> vertical
    for i in range(M):
        for j in range(N):
            
            #original fill horizontal
            if i == 0:
                if j == 0: #first fill
                    sim_table[i][j] = 1000
                elif j == 1: #first insert
                    sim_table[i][j] = p["{0}".format(trace2[j])]
                else: #rest fill, base insert scores
                    sim_table[i][j] = ins_score["{0}/{1}".format(trace2[j], trace2[j-1])] + sim_table[i][j-1]
            
            #original fill vertical
            elif j == 0:
                if i == 0:#first fill
                    sim_table[i][j] = 1000
                elif i == 1:
                    sim_table[i][j] = p["{0}".format(trace1[i])]
                else: #rest fill, base is the opposite of insert scores
                    sim_table[i][j] = -1*ins_score["{0}/{1}".format(trace1[i], trace1[i-1])] + sim_table[i-1][j]
            
            elif trace1[i] == trace2[j]: #no changes
                sim_table[i][j] = sim_table[i-1][j-1]
            
            else: #substitution, insertion or deletion
                
                #determine the min
                op = np.argmax([sim_table[i-1][j], sim_table[i][j-1], sim_table[i-1][j-1]]) #in order, removal, insertion, substitution
                if op == 0:
                    sim_table[i][j] = -1 + sim_table[i-1][j]#-1*ins_score["{0}/{1}".format(trace2[j],trace1[i])] + sim_table[i-1][j] #removal
                elif op == 1:
                    sim_table[i][j] = ins_score["{0}/{1}".format(trace2[j],trace1[i])] + sim_table[i][j-1] #insertion
                elif op == 2:
                    sim_table[i][j] = s_score["{0}, {1}".format(trace1[i],trace2[j])] + sim_table[i-1][j-1] #substitution
                
    return sim_table[i][j] #final score
            

In [49]:
t1 = ["a","a","b","a","a","b","c","c"]
t2 = ["a","a","b","a","a","b","a","a"]
t3 = ["a","a","a","a","b","c","c","c","c","c"]
t4 = ["a","a","a"]
t5 = ["b","b","a","b"]
t6 = ["c","b","b","b","b","c","b","c","c","b","a","b","c","c","a","a","b","c"]
t7 = ["c","c","c","c","c","c","c","a","a","b"]
t8 = ["c","c","c","c"]
t9 = ["b","a","a","b","a","a","b","c"]
t10 = ["a","c","c","b","a","c","c","b"]
tset = [t1,t2,t3,t4,t5,t6,t7,t8,t9,t10]
ssc = sub_scores(tset)
insc = insert_scores(tset)
prob = rgl_prob(tset)

a = ["a","c","c","b","b","a","b","b","c","c"]
b = ["b","c","c","a","a","a","c","c","a","b","b"]
c = ["a","c","c","b","b","a","b","b","c","c"]
sim1 = calc_similarity(a,b,ssc,insc,prob)
sim2 = calc_similarity(a,c,ssc,insc,prob)
print(sim1)
print(sim2) #we'll have to account for the fact that 0 is a special case

992.8561183322523
1000.0


In [50]:
tset1 = [test1, test2, test3]
ssc1 = sub_scores(tset1)
insc1 = insert_scores(tset1)
prob1 = rgl_prob(tset1)

/home/kense/.local/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log2
  app.launch_new_instance()


In [51]:
sim3 = calc_similarity(test1, test2, ssc1, insc1, prob1)
sim4 = calc_similarity(test1, test3, ssc1, insc1, prob1)
print(sim3)
print(sim4)

420.0001200798307
-156.00760062508994


# indeed, trace1 should be closer to trace2 than it is to trace3!

# Next, we can consider things like defined sub-conversations (trends of labels that are predetermined) and their frequencies. The intuition being that traces with similar label occurrences only address short term connections rather than long term trends, adding freq for long term trends should also assist in telling us if the "genre" or the "flow" of two traces are similar, which is also a valuable metric

In [54]:
def calc_similarity2(trace1, trace2, sub_cost, ins_cost, probs):
    
    assert type(trace1) == type(trace2) == list
    
    #pad traces
    trace1 = ["_"] + trace1
    trace2 = ["_"] + trace2

    M = len(trace1)
    N = len(trace2)
    sim_table = np.zeros((M,N)) #establish table
    s_score = sub_cost #get substitution score
    ins_score = ins_cost #get insertion score
    p = probs #get probabilities
    
    #fill table, horizontal -> vertical
    for i in range(M):
        for j in range(N):
            
            #original fill horizontal
            if i == 0:
                if j == 0: #first fill
                    sim_table[i][j] = 1000
                elif j == 1: #first insert
                    sim_table[i][j] = p["{0}".format(trace2[j])]
                else: #rest fill, base insert scores
                    sim_table[i][j] = ins_score["{0}/{1}".format(trace2[j], trace2[j-1])] + sim_table[i][j-1]
            
            #original fill vertical
            elif j == 0:
                if i == 0:#first fill
                    sim_table[i][j] = 1000
                elif i == 1:
                    sim_table[i][j] = p["{0}".format(trace1[i])]
                else: #rest fill, base is the opposite of insert scores
                    sim_table[i][j] = -1*ins_score["{0}/{1}".format(trace1[i], trace1[i-1])] + sim_table[i-1][j]
            
            elif trace1[i] == trace2[j]: #no changes
                sim_table[i][j] = sim_table[i-1][j-1]
            
            else: #substitution, insertion or deletion
                
                #determine the min
                op = np.argmax([sim_table[i-1][j], sim_table[i][j-1], sim_table[i-1][j-1]]) #in order, removal, insertion, substitution
                if op == 0:
                    sim_table[i][j] = -1 + sim_table[i-1][j]#-1*ins_score["{0}/{1}".format(trace2[j],trace1[i])] + sim_table[i-1][j] #removal
                elif op == 1:
                    sim_table[i][j] = ins_score["{0}/{1}".format(trace2[j],trace1[i])] + sim_table[i][j-1] #insertion
                elif op == 2:
                    sim_table[i][j] = s_score["{0}, {1}".format(trace1[i],trace2[j])] + sim_table[i-1][j-1] #substitution
                
    return sim_table[i][j] #final score

In [55]:
sim3 = calc_similarity2(test1, test2, ssc1, insc1, prob1)
sim4 = calc_similarity2(test2, test1, ssc1, insc1, prob1)
sim5 = calc_similarity2(test1, test3, ssc1, insc1, prob1)
sim6 = calc_similarity2(test3, test1, ssc1, insc1, prob1)
print(sim3)
print(sim4)
print(sim5)
print(sim6)

585.5541655852008
420.0001200798307
-156.00760062508994
1366.7683979683766


In [59]:
print(((1000-sim3)+(1000-sim4))/2)
print(((1000-sim5)+(sim6-1000))/2)

497.2228571674842
761.3879992967333


In [60]:
print(len(test1), len(test2), len(test3))

1074 939 1322
